In [74]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from godunov import *
from godunov_mod import *

def traffic_compute(N,T,cfl,rhom,um,godunov_compute,filename): 
    # Variables para facelas chamadas:
    #N=100 # INPUT
    #T=0.2 #min# INPUT
    #cfl=0.95# INPUT
    #rhom=750 #coches/km # INPUT
    #um=50 #  km/h
    #jam =0 # jam or not
    #filename='initial_condition_jam_horreo.mat'

    # Load rho0 and define w0 from filename
    content = sio.loadmat(filename)
    rho0 = content['rho0'][0]
    a = content['a'][0][0]
    b = content['b'][0][0]
    if(np.size(rho0)==1):
        w0=lambda x: rho0*np.ones(np.size(x))
    else:
        w0=lambda x: rho0

    # Parámetros para a representación:
    extrax=(b-a)/10.
    extray=rhom/10.
    liminf=0.
    limsup=rhom+extray

    # Solución aproximada
    if jam == 0:
        x,t,rho_aprox=godunov(a,b,N,T*60,cfl,w0,um*1000/3600,rhom/1000)
    else:
        x,t,rho_aprox=godunov_mod(a,b,N,T*60,cfl,w0,um*1000/3600,rhom/1000)

    Mt=np.size(t)-1
    dt=t[1]-t[0]
    rho_aprox[0,:]=np.nan
    rho_aprox[-1,:]=np.nan


In [76]:
%matplotlib inline
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 

plt.ioff()
fig, ax = plt.subplots()
ax.set_title('t = 0 min')
ax.set_xlabel('$x$ [m]')
ax.set_ylabel(r'$\rho$ [vehicles/km]')
line, = ax.plot([], [], lw=2)
out=widgets.Output()
style = {'description_width': 'initial'}

rhom=widgets.BoundedFloatText(
    min=0,
    max=10000.0,
    step=1.,
    description='Maximum density [vehicles/km]:',
    disabled=False, style=style
)

um=widgets.BoundedFloatText(
    min=0,
    max=100.0,
    step=1.,
    description='Maximum velocity [km/h]:',
    disabled=False, style=style
)

T=widgets.BoundedFloatText(
    min=0,
    max=5.0,
    step=0.1,
    description='Final time [min]:',
    disabled=False, style=style
)

filename = widgets.Dropdown(
    options=[('Jam at Horreo', 'initial_condition_jam_horreo.mat'), ('Jam at Ronda', 'initial_condition_jam_ronda.mat'), \
             ('100 vehicles at Ronda', 'initial_condition_uniform_100vechicles_ronda.mat'), ('140 vehicles at Horreo', \
              'initial_condition_uniform_140vechicles_horreo.mat')],
    value='initial_condition_jam_horreo.mat',
    description='Initial condition:', style=style
)

jam=widgets.RadioButtons(
    options=[('No',0), ('Yes',1)],
    description='Traffic interruption at x=0:',
    disabled=False, style=style
)

N=widgets.BoundedIntText(
    min=0,
    max=1000.,
    step=1,
    description='Number of cells:',
    disabled=False, style=style
)

cfl=widgets.BoundedFloatText(
    min=0,
    max=10.,
    step=0.01,
    description='CFL number:',
    disabled=False, style=style
)

button=widgets.Button(
    description='Run',
    disabled=False,
    button_style='',
    tooltip='Run',
    icon='check'
)

vbox=widgets.VBox(children=(out,rhom,um,T,filename,jam,N,cfl,button))
display(vbox)

def on_button_clicked(b):
    a,b,x,rho_aprox,extrax,extray,liminf,limsup,Mt = \
      traffic_compute(N.value,T.value,cfl.value,rhom.value,um.value,jam.value,filename.value)
    def init():
        ax.set_xlim(a-extrax,b+extrax)
        ax.set_ylim(liminf,limsup)
        line.set_data([], [])
        return (line,)
    
    def animate(i):
        # update the data
        line.set_data(x, 1000.*rho_aprox[:,i])
        return (line,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=Mt, interval=100, 
                               blit=True)
    HTML(anim.to_jshtml())
    
button.on_click(on_button_clicked)
